<a href="https://colab.research.google.com/github/seunghyeon98/NLP/blob/main/%08roberta_%EB%AC%B8%EC%9E%A5%EC%9C%A0%ED%98%95%EB%B6%84%EB%A5%98_finetuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 73.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 101.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 182.4/182.4 KB 21.4 MB/s eta 0:00:00


In [ ]:
%cd /content/drive/MyDrive/데이콘) 문장 유형 분류/데이콘 문장 유형 분류


/content/drive/MyDrive/데이콘) 문장 유형 분류/데이콘 문장 유형 분류


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# https://dacon.io/en/competitions/official/236037/overview/description

import pandas as pd
import numpy as np
import torch
import os
import random
from sklearn.model_selection import train_test_split
from transformers import AutoModel, AutoTokenizer
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from torch import nn
from tqdm import tqdm

# for graphing
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
train_original = pd.read_csv('train.csv')
train_original.drop(columns=['ID'],inplace=True)

test = pd.read_csv('test.csv')
test.drop(columns=['ID'],inplace=True)

submission = pd.read_csv('sample_submission.csv')

In [ ]:
test

,문장
0,"장욱진의 ＇가족＇은 허물 없는 가족애를, 처음 공개되는 정약용의 ＇정효자전＇과 ＇정..."
1,"조지 W 부시, 버락 오바마 전 대통령도 전쟁 위험 때문에 버린 카드다."
2,지난해 1분기 128억원이었던 영업이익이 올해 1분기 505억원으로 급증했다.
3,수상 작가와 맺으려던 계약서 내용 가운데 일부가 ＇독소 조항＇으로 해석돼 수정을 요...
4,결국 최근 KDB산업은행은 대규모 손실 위기에 닥친 에어부산에 140억원 금융지원을...
...,...
7085,"2020 세계국가편람 모바일 앱은 세계 216개국의 국가개황과 주요 경제지표, 사회..."
7086,탈세계화 징후들이 반갑지 않은 이유다.
7087,"틱톡은 6월 ＇인터넷 안전의 달＇을 맞아 올바른 개인정보 보호 관리 방법, 앱 내 ..."
7088,만약 3개월 간 채굴자들의 투표를 거쳐 2/3 이상의 해시파워가 ＇채굴세＇ 도입에 ...


In [ ]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True

CFG = {
    'EPOCHS':5,
    'LEARNING_RATE':1e-5,
    'BATCH_SIZE':32,
    'SEED':41
}

seed_everything(CFG['SEED']) # Seed 고정
device = torch.device('cuda')

In [ ]:
train,val,_,_ = train_test_split(train_original, train_original['label'], test_size=0.2,random_state=CFG['SEED'])
train.reset_index(drop=True,inplace=True)
val.reset_index(drop=True,inplace=True)


In [ ]:
train_original.isna().sum()

문장       0
유형       0
극성       0
시제       0
확실성      0
label    0
dtype: int64

In [ ]:
val

,문장,유형,극성,시제,확실성,label
0,장교 출신 예비역 최 모씨(29)는 ＂정당하게 병역의무를 다한 사람은 마치 비양심인...,사실형,긍정,과거,확실,사실형-긍정-과거-확실
1,배 대표는 ＂1998년 아버지에게 누룩 회사인 ＇한국발효＇를 받아 누룩 사업을 하다...,사실형,긍정,과거,확실,사실형-긍정-과거-확실
2,바이러스라는 전혀 예상치 못했던 돌발 변수 출현으로 투자자 모집과 거래 완료까지 소...,사실형,긍정,현재,확실,사실형-긍정-현재-확실
3,참가자들은 총 3개까지 출품 희망 부문을 복수 선택할 수 있다.,추론형,긍정,미래,불확실,추론형-긍정-미래-불확실
4,샤론 최씨도 ＂그렇다. 영화감독을 하고 싶다＂고 말하자 진행자들은 ＂내년 시상식에서...,사실형,긍정,과거,확실,사실형-긍정-과거-확실
...,...,...,...,...,...,...
3304,"이번 소집 명단에서 한국과 일본을 제외한 유럽파는 정우영(SC프라이부르크, 분데스리...",사실형,긍정,현재,확실,사실형-긍정-현재-확실
3305,"건선은 대개 팔꿈치, 무릎, 사지의 바깥 부분, 둔부, 두피 등 외부의 자극을 받기...",사실형,긍정,과거,확실,사실형-긍정-과거-확실
3306,다른 경찰관은 김창룡 경찰청장을 직접 언급하며 ＂(청장의) 잔여 임기 동안 경찰국 ...,사실형,긍정,과거,확실,사실형-긍정-과거-확실
3307,코로나19 이후 지속된 초저금리와 증시 부진이 발목을 잡았다.,사실형,긍정,과거,확실,사실형-긍정-과거-확실


In [ ]:
check_point = 'klue/roberta-small'
base_model = AutoModel.from_pretrained(check_point)
tokenizer = AutoTokenizer.from_pretrained(check_point)
# tokenizer_opt = tokenizer(
#     text = input_text,
#     padding = 'max_length',
#     truncation = True,
#     max_length = 90,
#     return_tensors ='pt'
# )


Downloading:   0%|          | 0.00/545 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/273M [00:00<?, ?B/s]

Some weights of the model checkpoint at klue/roberta-small were not used when initializing RobertaModel: ['lm_head.layer_norm.weight', 'lm_head.decoder.weight', 'lm_head.layer_norm.bias', 'lm_head.dense.weight', 'lm_head.decoder.bias', 'lm_head.bias', 'lm_head.dense.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaModel were not initialized from the model checkpoint at klue/roberta-small and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it f

Downloading:   0%|          | 0.00/375 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/248k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/752k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/173 [00:00<?, ?B/s]

In [ ]:
class SentenceTypeDataset(Dataset):
  def __init__(self,dataframe,tokenizer,labels=None):
    # x를 리스트로만듬
    texts = dataframe['문장'].values.tolist()
    # 토크나이징
    self.texts = [tokenizer(text, padding='max_length', max_length=90,truncation=True,return_tensors='pt') for text in texts]
    self.labels = labels

  def __len__(self):
    return len(self.texts)

  def __getitem__(self,idx):
    text = self.texts[idx]
    if self.labels is not None:
      type_tmp = self.labels['type'][idx]
      return text, torch.Tensor(type_tmp)
    else:
      return text, torch.Tensor([-1,-1,-1,-1])


In [ ]:
class SentenceClf(nn.Module):
  def __init__(self,base_model):
    super().__init__()
    self.klue = base_model

    self.fc1 = nn.Linear(768,32)
    self.relu = nn.ReLU()

    self.type_clf = nn.Linear(32,4)
    self.softmax = nn.Softmax(dim=1)

  def forward(self,input_ids,attention_mask):
    klue_out = self.klue(input_ids = input_ids, attention_mask = attention_mask)[0][:,0]

    x = self.fc1(klue_out)
    x = self.relu(x)

    type_output = self.type_clf(x)
    type_output = self.softmax(type_output)

    return type_output

In [ ]:
def sentence_train(model, train_dataloader, val_dataloader, learning_rate, epochs, model_nm):
    best_val_loss = 99999999999999 # setting max (act as infinity)
    early_stopping_threshold_count = 0

    criterion = {
        'type' : nn.CrossEntropyLoss().to(device),
    }

    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
    model = model.to(device)

    for epoch in range(epochs+1):
        total_acc_train = 0
        total_loss_train = 0

        model.train() # sets into the training mode

        for train_input, type_label in tqdm(train_dataloader):
            attention_mask = train_input['attention_mask'].to(device)
            input_ids = train_input['input_ids'].squeeze(1).to(device)
            type_label = type_label.to(device)

            optimizer.zero_grad()

            type_output = model(input_ids, attention_mask) # from the forward function

            # 4개의 label을 한꺼번에 업데이트하니 0.25를 한듯
            loss = criterion['type'](type_output, type_label.float())

            # 4개의 label의 loss를 다 합쳐서 total_loss에 담는다
            total_loss_train += loss.item()

            # 미분계산
            loss.backward()
            # 업데이트
            optimizer.step()


        with torch.no_grad(): # since we should not change gradient for validation
            total_acc_val = 0
            total_loss_val = 0

            model.eval() # deactivate training

            # same process as the above
            for val_input, vtype_label in tqdm(val_dataloader):
                attention_mask = val_input['attention_mask'].to(device)
                input_ids = val_input['input_ids'].squeeze(1).to(device)

                vtype_label = vtype_label.to(device)

                vtype_output = model(input_ids, attention_mask) # from the forward function

                loss = criterion['type'](vtype_output, vtype_label.float())

                total_loss_val += loss.item()


            print(f'Epochs: {epoch + 1} '
                  f'| Train Loss: {total_loss_train / len(train_dataloader): .3f} '
                  f'| Train Accuracy: {total_acc_train / (len(train_dataloader.dataset)): .3f} '
                  f'| Val Loss: {total_loss_val / len(val_dataloader): .3f} '
                  f'| Val Accuracy: {total_acc_val / len(val_dataloader.dataset): .3f}')

            if best_val_loss > total_loss_val:
                best_val_loss = total_loss_val # saving only the best one
                torch.save(model, f"./{model_nm}.pt")
                print("Saved model")
                early_stopping_threshold_count = 0
            else:
                early_stopping_threshold_count += 1 # checking how many epochs have passed that val_loss didn't increase

            if early_stopping_threshold_count >= 3: # ==> patience=1
                print("Early stopping")
                break

In [ ]:
train_tmp = train[['문장','유형']]
train_tmp = pd.get_dummies(train_tmp,columns=['유형'])
train_tmp

,문장,유형_대화형,유형_사실형,유형_예측형,유형_추론형
0,용산구청 관계자는 ＂재정이 열악한 지자체로서는 1800억원을 마련할 수 없다＂며 서...,0,1,0,0
1,부산시는 이처럼 부산이 가파른 상승세를 보이는 이유에 대해 지난해부터 추진하고 있는...,0,1,0,0
2,"그러나 미숙아, 만성호흡기질환, 선천 심장병, 선천 면역결핍질환, 암환자 등의 고위...",0,1,0,0
3,탁구 종목에서 중국 대표팀 위상이 뛰어나기 때문이다.,0,0,0,1
4,이 논문에 따르면 ＇BT-11＇은 뇌의 신경전달물질인 아세틸콜린을 분해하는 효소의 ...,0,1,0,0
...,...,...,...,...,...
13227,우리가 익히 아는 대로 임꺽정은 신출귀몰했다.,0,1,0,0
13228,김 상무보는 ＂실제 이용자 수와 인당 사용시간 등 주요 데이터가 매년 두 자릿수 상...,0,1,0,0
13229,＇디폴트 옵션＇의 필요성을 주장해온 쪽이 항상 사례로 들어온 것이 ＇401K＇로 불...,1,0,0,0
13230,1992년부터 선양시 조선족노인협회를 후원하기 시작해 1997년에는 1500㎡ 건물...,0,1,0,0


In [ ]:
train_type = train_tmp.iloc[:,1:5].values.tolist()
train_labels = {
    'type' : train_type
}

In [ ]:
val_tmp = val[['문장','유형']]
val_tmp = pd.get_dummies(val_tmp,columns=['유형'])
val_type = val_tmp.iloc[:,1:5].values.tolist()
val_labels = {
    'type' : val_type
}

In [ ]:
train_tmp

,문장,유형_대화형,유형_사실형,유형_예측형,유형_추론형
0,용산구청 관계자는 ＂재정이 열악한 지자체로서는 1800억원을 마련할 수 없다＂며 서...,0,1,0,0
1,부산시는 이처럼 부산이 가파른 상승세를 보이는 이유에 대해 지난해부터 추진하고 있는...,0,1,0,0
2,"그러나 미숙아, 만성호흡기질환, 선천 심장병, 선천 면역결핍질환, 암환자 등의 고위...",0,1,0,0
3,탁구 종목에서 중국 대표팀 위상이 뛰어나기 때문이다.,0,0,0,1
4,이 논문에 따르면 ＇BT-11＇은 뇌의 신경전달물질인 아세틸콜린을 분해하는 효소의 ...,0,1,0,0
...,...,...,...,...,...
13227,우리가 익히 아는 대로 임꺽정은 신출귀몰했다.,0,1,0,0
13228,김 상무보는 ＂실제 이용자 수와 인당 사용시간 등 주요 데이터가 매년 두 자릿수 상...,0,1,0,0
13229,＇디폴트 옵션＇의 필요성을 주장해온 쪽이 항상 사례로 들어온 것이 ＇401K＇로 불...,1,0,0,0
13230,1992년부터 선양시 조선족노인협회를 후원하기 시작해 1997년에는 1500㎡ 건물...,0,1,0,0


In [ ]:
train_dataloader = DataLoader(SentenceTypeDataset(train_tmp, tokenizer, train_labels), batch_size=CFG['BATCH_SIZE'], shuffle=True, num_workers=0) # num_workers: how many subprocesses to use for data loading
val_dataloader = DataLoader(SentenceTypeDataset(val_tmp, tokenizer, val_labels), batch_size=CFG['BATCH_SIZE'], num_workers=0)

In [ ]:
model = SentenceClf(base_model)

In [ ]:
sentence_train(model, train_dataloader, val_dataloader,CFG['LEARNING_RATE'],CFG['EPOCHS'],'kclue_type')

100%|██████████| 104/104 [00:07<00:00, 13.12it/s]


Epochs: 1 | Train Loss:  0.208 | Train Accuracy:  0.000 | Val Loss:  0.214 | Val Accuracy:  0.000
Saved model


100%|██████████| 104/104 [00:07<00:00, 13.22it/s]


Epochs: 2 | Train Loss:  0.207 | Train Accuracy:  0.000 | Val Loss:  0.214 | Val Accuracy:  0.000
Saved model


100%|██████████| 104/104 [00:07<00:00, 13.23it/s]


Epochs: 3 | Train Loss:  0.206 | Train Accuracy:  0.000 | Val Loss:  0.214 | Val Accuracy:  0.000


100%|██████████| 104/104 [00:07<00:00, 13.37it/s]


Epochs: 4 | Train Loss:  0.206 | Train Accuracy:  0.000 | Val Loss:  0.214 | Val Accuracy:  0.000
Saved model


100%|██████████| 104/104 [00:07<00:00, 13.40it/s]

Epochs: 5 | Train Loss:  0.205 | Train Accuracy:  0.000 | Val Loss:  0.215 | Val Accuracy:  0.000


In [ ]:
def get_type_predictions(model,loader):
  device = torch.device('cuda')
  model = model.to(device)

  type_probs = []

  with torch.no_grad():
    model.eval()
    for data_input,_ in tqdm(loader):

      attention_mask = data_input['attention_mask'].to(device)

      # 왜 squeeze(1)를 하는가. 왜 1번째의 차원을 줄이는가
      input_ids = data_input['input_ids'].squeeze(1).to(device)

      type_output = model(input_ids, attention_mask)

      type_probs.append(type_output)

      return torch.cat(type_probs).cpu().detach().numpy()

In [ ]:
model = torch.load('./kclue_type.pt')

In [ ]:
test_dataloader = DataLoader(SentenceTypeDataset(test,tokenizer),batch_size=CFG['BATCH_SIZE'],shuffle=False)

In [ ]:
test_pred_type = get_type_predictions(model, test_dataloader)

  0%|          | 0/222 [00:00<?, ?it/s]


In [ ]:
# print(test_pred_type[0])
# print(np.argmax(test_pred_type[0]))
print(len(test_pred_type))

32


In [ ]:

test_type = ['대화형' if i==0 else '사실형' if i==1 else '예측형' if i==2 else '추론형' for i in [np.argmax(p) for p in test_pred_type]]


In [ ]:
label_sum = []
for i in range(len(test_type)):
  label_sum.append(f'{test_type[i]}')

submission['label'] = label_sum
submission

ValueError: ignored

In [ ]:
submission['pred_type'] = label_sum

ValueError: ignored

In [ ]:
len(label_sum)

32